# SRT Lab: The Sun's Brightness Temperature

In [ ]:
import digital_rf
from digital_rf.digital_metadata import DigitalMetadataReader
from drf_process import *
from astropy.io import fits
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os
from dataprocessing import *

## Option 1: Read in Digital RF Data

In [ ]:
#Run this block if your data is in digital rf format
#If the data is discontinuous (you get errors trying to read it) it may be necessary to find the ranges
#in which there is data. If so, uncomment the call to getranges(), filling in the location of your metadata, 
#and uncomment the instantatiation of an SRTData object that uses those ranges as an argument
def getranges(metadatadir,outfile):
    """
    metadatadir: str, the directory of your metadata
    outfile: str, the .txt file to write the data ranges to
    """
    meta = DigitalMetadataReader(metadatadir, accept_empty=False)
    metadf = meta.read_dataframe(meta.get_bounds()[0], meta.get_bounds()[1])
    find_boundset(metadf, outfile)
    f = open(outfile,'r')
    r = [line.replace('\n','') for line in f]
    ranges = []
    for pair in r:
        ranges.append((int(pair.split(' ')[0]),int(pair.split(' ')[1])))
    print(ranges)
    return ranges
ranges= getranges('srtlabs/srt-azel-test/ch1','testing.txt')
srtdata = SRTData(datadir=os.path.join(os.path.dirname(os.path.abspath(os.curdir)),'srtlabs/srt-azel-test/ch1'),
                    params=["power","alt", "az"],ranges = ranges).process()
#Uncomment below if data is continuous (no ranges necessary)
#srtdata = SRTData(datadir=os.path.join(os.path.dirname(os.path.abspath(os.curdir)),'srtlabs/srt-azel-test/ch1'),
                    #params=["power","alt"]).process()

## Option 2 (Recommended, Faster): Read in .fits Data

In [ ]:
azfile = 'srt-azel-test/az.fits' 
elfile = 'srt-azel-test/el.fits'
azscan = pd.DataFrame({'power':[],'alt':[],'az':[]})
elscan = pd.DataFrame({'power':[],'alt':[],'az':[]})
files = [azfile, elfile]
dfs = [azscan, elscan]
for z in range(len(files)):
    hdul = fits.open(files[z])
    az = []
    el = []
    values = []
    data = hdul[0].data/len(hdul)
    for i in range(1,len(hdul)):
        metadata = hdul[i].header['METADATA'].split(',')
        data = data + hdul[i].data/len(hdul)
        values.append(np.mean(data))
        for element in metadata:
            if "motor_el" in element:
                el.append(float(element.split(':')[-1]))
            elif "motor_az" in element:
                az.append(float(element.split(':')[-1]))
    dfs[z]['power'] = values
    dfs[z]['alt'] = el
    dfs[z]['az'] = az

## Npoint Scan (For Digital RF)

In [ ]:
# make contour plot
fig = go.Figure()
x, y = np.asarray(alt), np.asarray(az)
z = np.asarray(srtdata["power"]).reshape((5, 5))
fig = go.Figure(data=go.Contour(z=z, x=x, y=y))
fig.update_layout(
    title="5x5 Npoint Scan",
    xaxis_title="Altitude (degrees)",
    yaxis_title="Azimuth (degrees)",
)
fig.show()

## Azimuth/Altitude Scan (For .fits)

In [ ]:
# Altitude Scan
fig = go.Figure()
alt = np.asarray(elscan['alt'])
power = np.asarray(elscan["power"])
fig = go.Figure(data=go.Scatter(x=alt, y=power, mode="markers"))
fig.update_layout(
    title="Altitude Scan", xaxis_title="Altitude (degrees)", yaxis_title="Power",
)
fig.show()

In [ ]:
# Azimuth Scan
# Note to self -- look up how to convert power to temp
fig = go.Figure()
az = np.asarray(azscan['az'])
power = np.asarray(azscan["power"])
fig = go.Figure(data=go.Scatter(x=az, y=power, mode="markers"))
fig.update_layout(
    title="Azimuth Scan", xaxis_title="Azimuth (degrees)", yaxis_title="Power",
)
fig.show()